In [58]:
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from os import urandom

# Generate a random salt for PBKDF2 key derivation function
salt = urandom(16)

# Derive a 256-bit encryption key using PBKDF2
password = b"mysecretkey"
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000,
)
key = kdf.derive(password)

# Encrypt a message
nonce = urandom(12)

print('key : ' + str(key))
print('nonce : ' + str(nonce))

key : b'\x1d\x8cG\xb8\t\xb5\xed\xd7\xda\xcf\x90%~\x1a\xa9\xc8\x94\xddt\x94\x08\xe9\x91\xa1\x88\x10\x8c\x82\xcc~\x9bA'
nonce : b'\xd4\x0c4\xd7\xb9^\x8cu\xe4\x92\x8c\x0b'


In [60]:
with open("keyfile.bin", "wb") as f:
    f.write(key)
    f.write(nonce)


In [61]:
# Load the key and nonce from the file
with open("keyfile.bin", "rb") as f:
    key = f.read(32)
    nonce = f.read(12)

In [65]:
id = 3
plaintext = id.to_bytes(16, byteorder='big')
cipher = Cipher(algorithms.AES(key), modes.GCM(nonce))
encryptor = cipher.encryptor()
ciphertext = encryptor.update(plaintext) + encryptor.finalize()
tag = encryptor.tag
print(tag)

b'A\xa6\x89\xbf\x1e\xe4\xe4\xc4b\xbaZ\x04\x87\xc2\x8fo'


In [ ]:
# Decrypt the ciphertext
cipher = Cipher(algorithms.AES(key), modes.GCM(nonce, tag))
decryptor = cipher.decryptor()
decrypted_plaintext = decryptor.update(ciphertext) + decryptor.finalize()

# Print the results
print(f"Key: {key}")
print(f"Nonce: {nonce}")
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted plaintext: {decrypted_plaintext}")

# Encrypt the same plaintext again with the same key and nonce
cipher = Cipher(algorithms.AES(key), modes.GCM(nonce))
encryptor = cipher.encryptor()
ciphertext2 = encryptor.update(plaintext) + encryptor.finalize()
tag2 = encryptor.tag

# Decrypt the second ciphertext using the same nonce and key
cipher = Cipher(algorithms.AES(key), modes.GCM(nonce, tag2))
decryptor = cipher.decryptor()
decrypted_plaintext2 = decryptor.update(ciphertext2) + decryptor.finalize()

# Print the second ciphertext and decrypted plaintext
print(f"Ciphertext 2: {ciphertext2}")
print(f"Decrypted plaintext 2: {decrypted_plaintext2}")

In [66]:
import hashlib
import base64

def num_to_random_string(num):
    # Convert number to bytes
    num_bytes = str(num).encode('utf-8')

    # Generate hash value
    hash_object = hashlib.sha256(num_bytes)
    hash_bytes = hash_object.digest()

    # Encode hash value as base64 string
    random_string = base64.b64encode(hash_bytes).decode('utf-8')

    return random_string

In [73]:
num_to_random_string("fzfezfz")

'TNcr8D0FB0K+bNoK4DVLx3qXvh9LgRf+iTGhEUWY9fg='